Notebook to experiment with the following end to end process: from dataset+task in NL, to typology based diagram, to design recommendation.

In [2]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import Ollama
from langchain.text_splitter import TokenTextSplitter
import glob
from pprint import pprint

from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

from langchain.schema.runnable import RunnableMap
from langchain_core.prompts import PromptTemplate

In [3]:
model = Ollama(model="llama3:8b", temperature=0)

## Determine the end goal/decision
Input: dataset description

Output: The decision task

In [4]:
documents = []
documents.extend(PyPDFLoader("docs/dm.pdf").load())

In [5]:
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=25)
docs = text_splitter.split_documents(documents)
len(docs)

260

In [6]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

persist_directory = 'docs/chroma/'
!rm -rf docs/chroma  # remove old database files if any

vectordb = Chroma.from_documents( # had an error previuously, downgraded to chromadb version 0.4.3 using command: pip install chromadb==0.4.3. See https://github.com/zylon-ai/private-gpt/issues/1012
    documents=docs,
    embedding=hf,
    persist_directory=persist_directory,
)
retriever = vectordb.as_retriever()

print(vectordb._collection.count())

/Users/cameliadanielabrumar/anaconda3/envs/llama-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/cameliadanielabrumar/anaconda3/envs/llama-env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


260


In [7]:
nl_to_typology_goal_template = """Imagine you are a visualization designer who wants to understand what are the decisions an expert in embryology and in vitro fertilization would make when designing a visualization.
You are tasked with taking the dataset and the task the expert is trying to accomplish and translating the task into one of the decision making tasks that appear in Typology of
Decision-Making Tasks for Visualization paper.

The three possible decision tasks are: CHOOSE, ACTIVATE and CREATE. Give a brief explanation of the decision making task you chose and why you think it is the most appropriate for the task at hand.
When providing reasons, give explanations that relate to the definitions of the three tasks as described in the Typology of Decision-Making Tasks for Visualization paper.
The dataset description, task description, and typology of decision making tasks paper are given below. 

Relevant context from the Typology of Decision-Making Tasks for Visualization paper: {context}
Data Description: {data_description}
Task Description: {task_description}
"""
# prompt = ChatPromptTemplate.from_template(template)
nl_to_typology_goal_prompt_template = PromptTemplate.from_template(nl_to_typology_goal_template)

In [8]:
dm_task_definitions_question = "What are the three decision making tasks in my typology?"
retriever.get_relevant_documents(dm_task_definitions_question)

/Users/cameliadanielabrumar/anaconda3/envs/llama-env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content=' a typology for decision-making tasks in visualiza-\ntion, addressing the limitations of existing taxonomies. Built upon prior\nresearch and informed by design goals derived from a thorough liter-\nature review, the typology comprises three decision tasks: CHOOSE,\nACTIV ATE, and CREATE. These tasks allow for the representation\nof complex decision-making structures, as they can be composed or\ndecomposed into other tasks. The typology demonstrates completeness,', metadata={'page': 8, 'source': 'docs/dm.pdf'}),
 Document(page_content=' real-world visualization\nsystems.\n4.1 Decision-Making Tasks\nOur typology consists of three tasks derived from the scientific\nliterature [27, 28] : CHOOSE, ACTIV ATE, and CREATE. Each task is\na function that represents a specific and distinct decision problem. The\ntype of the inputs to these functions does not change the core process\nof the decision task. Some of the key differences between the tasks are\nthe unique transfor

In [9]:
nl_to_typology_goal_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(dm_task_definitions_question),
    "data_description": lambda x: x["data"],
    "task_description": lambda x: x["task"]
}) | nl_to_typology_goal_prompt_template | model

In [10]:
nl_to_typology_goal_chain_output =  nl_to_typology_goal_chain.invoke({"data": "tabular data where each row is a patient, and the associated levels of age, bmi amh and afc at the time of the Egg Retrieval Procedure.",
              "task": "understand how the medication dose varies with the following patient parameters: age, bmi amh and afc then recommend a dosage for the current patient."
              })
print(nl_to_typology_goal_chain_output)

After analyzing the dataset and task description, I believe that the most appropriate decision-making task is ACTIVATE.

The ACTIVATE task represents a decision where options are evaluated, and only those that meet or exceed a threshold are returned. In this case, the expert in embryology and in vitro fertilization needs to evaluate the medication dose based on various patient parameters (age, BMI, AMH, and AFC) and recommend a dosage for the current patient.

The task requires evaluating options (different medication doses) against specific criteria (patient parameters), and only those that meet or exceed a certain threshold (optimal dosage) are returned. This process involves filtering out suboptimal options based on the evaluation of the patient's characteristics, which aligns with the ACTIVATE task definition.

In contrast, the CHOOSE task would require selecting one option from a set of available options, which might not accurately capture the complexity of evaluating multiple pat

## Expand the decision tasks iteratively
Input: The  decision task.

Ouput: a typology based diagram.

In [11]:
decision_task = model.invoke("What is the decision task the following text is describing? Answer with one of these three words and nothing else: CHOOSE, ACTIVATE or CREATE. \n" 
                             + nl_to_typology_goal_chain_output)

decision_task

'ACTIVATE'

In [12]:
decomposition_context_question = "How do you decompose decision making tasks?"
retriever.get_relevant_documents(decomposition_context_question)

[Document(page_content=', during our interview study, we observed a preference among\nparticipants for a top-down approach (see Section 7). The participantsstarted with high-level decision tasks and recursively decomposed them\ninto sub-tasks until they achieved the desired level of granularity. In the\ncase studies below, we demonstrate how these decision-task hierarchies\ncan be created through a series of decompositions.\n6 C ASESTUDIES\nTo illustrate the composability andh', metadata={'page': 4, 'source': 'docs/dm.pdf'}),
 Document(page_content=' of real-world decision-making\nproblems, as our tasks can be composed or decomposed into other\ntasks.arXiv:2404.08812v2  [cs.HC]  22 Apr 2024', metadata={'page': 0, 'source': 'docs/dm.pdf'}),
 Document(page_content=' This involved dissecting the decision processes, identifying key\ncomponents, and highlighting other properties relevant to decision-\nmaking, as outlined in subsection 3.3.\nFollowing this, we conducted collaborative working

In [36]:
typology_goal_to_diagram_template = """Imagine you are a visualization designer who wants to understand what are the decisions an expert in embryology and in vitro fertilization would make when recommending medication for ovarian stimulation.
You are tasked with taking the decision goal explained as context below, and expanding it according to the decomposition instructions, also given below. The decision goal should be decomposed
in {number_of_subtasks} decision subtasks. Each subtask should be also one of the three decision making tasks that appear in Typology of Decision-Making Tasks for Visualization paper (CHOOSE, ACTIVATE, CREATE).
Explain how the decisions are connected to each other and how they contribute to the overall decision goal. 
For that, describe the node-link diagram of the decision making process in a couple of tables, where each node is a decision subtask and each edge is a connection between two decision subtasks.
The first table has the following columns: node id, node type (CHOOSE, ACTIVATE, or CREATE), that is, the name of the decision task, and the description of the decision task. The node id starts at 1, and increments by 1 for each decision task.
The second table has the following columns: Source, Target, and the information being passed from one decition to the other. 
The Source and Target columns should contain the ids of the decisiont tasks, names of the decision tasks, and the Description column should contain a brief explanation of the connection between the two decision tasks.

Don't forget to add what the decision goal is at the beginning of the response.

Relevant context from the Typology of Decision-Making Tasks for Visualization paper: {decision_task_definitions}
Relevant decomposition instructions from the Typology of Decision-Making Tasks for Visualization paper: {decomposition_context}
Decision Goal Description: {decision_goal_description}
"""
# prompt = ChatPromptTemplate.from_template(template)
typology_goal_to_diagram_prompt_template = PromptTemplate.from_template(typology_goal_to_diagram_template)

In [37]:
typology_goal_to_diagram_chain = RunnableMap({
    "number_of_subtasks": lambda x: x["number_of_subtasks"],
    "decision_task_definitions": lambda x: retriever.get_relevant_documents(dm_task_definitions_question),
    "decomposition_context": lambda x: retriever.get_relevant_documents(decomposition_context_question),
    "decision_goal_description": lambda x: x["decision_goal_description"]
}) | typology_goal_to_diagram_prompt_template | model

In [38]:
typology_goal_to_diagram_chain_output =  typology_goal_to_diagram_chain.invoke({"number_of_subtasks": 2,
              "decision_goal_description": nl_to_typology_goal_chain_output
              })
print(typology_goal_to_diagram_chain_output)

**Decision Goal:** After analyzing the dataset and task description, I believe that the most appropriate decision-making task is ACTIVATE.

**Decomposition of Decision Goal:**

| Node ID | Node Type | Description |
| --- | --- | --- |
| 1 | CHOOSE | Evaluate patient parameters (age, BMI, AMH, AFC) to determine relevant medication options. |
| 2 | ACTIVATE | Activate the evaluation process by comparing each medication option against the patient's parameters and filter out suboptimal options based on a threshold (optimal dosage). |

**Node-Link Diagram:**

| Source | Target | Description |
| --- | --- | --- |
| 1 | 2 | The CHOOSE task provides the relevant patient parameters, which are used to evaluate medication options in the ACTIVATE task. |
| 2 | - | The ACTIVATE task outputs a filtered list of optimal medication options based on the evaluation process. |

In this decomposition, the CHOOSE task (Node 1) is responsible for evaluating patient parameters and determining relevant medicat

Note to self: this can be improved by adding memory to the llm, that is, I shouldn't have to pass it the previous output, to every new prompt template, but I should just use a template.
See the memory module in this course: https://learn.deeplearning.ai/courses/langchain/lesson/3/memory

From natural language to pandas diagram. See example of how to format the graph in a table here: https://towardsdatascience.com/visualizing-networks-in-python-d70f4cbeb259

In [86]:
# testing the json parser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define your desired data structure.
# took the structure from here: https://gist.github.com/mbostock/4062045
class Node(BaseModel):
    node_id: int = Field(description="this field is called 'id'. id of the node in the graph, starting at 1")
    node_type: dict = Field(description="this field is called 'node_type'. node in the graph with the name of the associated decision making task (CHOOSE, ACTIVATE or CREATE)")
    node_description: dict = Field(description="this field is called 'description'. description of the decision making task")

class Link(BaseModel):
    source: dict = Field(description="this field is called 'source'. id of the source node in the graph")
    target: dict = Field(description="this field is called 'target'. id of the target node in the graph")

class Graph(BaseModel):
    nodes: list = Field(description="list of nodes in the graph")
    links: list = Field(description="list of links in the graph")



In [93]:
# And a query intented to prompt a language model to populate the data structure.
query = '''Parse the following decision process in a graph. The decisions are nodes, and the dependencies between the decisions are links. 
Make sure that the nodes have an id and that the links use the actual node ids. 
The decision process is described as follows: \n ''' + typology_goal_to_diagram_chain_output

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Graph)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

output_json = chain.invoke({"query": query})

In [94]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"nodes": {"title": "Nodes", "description": "list of nodes in the graph", "type": "array", "items": {}}, "links": {"title": "Links", "description": "list of links in the graph", "type": "array", "items": {}}}, "required": ["nodes", "links"]}\n```'

In [95]:
output_json

{'properties': {'nodes': {'title': 'Nodes',
   'description': 'list of nodes in the graph',
   'type': 'array',
   'items': [{'id': 1,
     'type': 'CHOOSE',
     'description': 'Evaluate patient parameters (age, BMI, AMH, AFC) to determine relevant medication options.'},
    {'id': 2,
     'type': 'ACTIVATE',
     'description': "Activate the evaluation process by comparing each medication option against the patient's parameters and filter out suboptimal options based on a threshold (optimal dosage)."}]},
  'links': {'title': 'Links',
   'description': 'list of links in the graph',
   'type': 'array',
   'items': [{'source': 1,
     'target': 2,
     'description': 'The CHOOSE task provides the relevant patient parameters, which are used to evaluate medication options in the ACTIVATE task.'}]}},
 'required': ['nodes', 'links']}

In [96]:
output_json['properties']['nodes']['items']

[{'id': 1,
  'type': 'CHOOSE',
  'description': 'Evaluate patient parameters (age, BMI, AMH, AFC) to determine relevant medication options.'},
 {'id': 2,
  'type': 'ACTIVATE',
  'description': "Activate the evaluation process by comparing each medication option against the patient's parameters and filter out suboptimal options based on a threshold (optimal dosage)."}]

In [97]:
import pandas as pd

nodes_df = pd.read_json(json.dumps(output_json['properties']['nodes']['items']))
nodes_df

/var/folders/xq/gb72q1px7yb_s35rx4h5l0n00000gn/T/ipykernel_28396/3712040430.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  nodes_df = pd.read_json(json.dumps(output_json['properties']['nodes']['items']))


,id,type,description
0,1,CHOOSE,"Evaluate patient parameters (age, BMI, AMH, AF..."
1,2,ACTIVATE,Activate the evaluation process by comparing e...


In [98]:
edges_df = pd.read_json(json.dumps(output_json['properties']['links']['items']))
edges_df

/var/folders/xq/gb72q1px7yb_s35rx4h5l0n00000gn/T/ipykernel_28396/1606649814.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  edges_df = pd.read_json(json.dumps(output_json['properties']['links']['items']))


,source,target,description
0,1,2,The CHOOSE task provides the relevant patient ...


In [105]:
G = nx.from_pandas_edgelist(edges_df, 'source', 'target', 'description')

In [106]:
node_attributes = nodes_df.set_index('id')['type'].to_dict()
nx.set_node_attributes(G, node_attributes, 'node_type')

In [108]:
G[1]

AtlasView({2: {'description': 'The CHOOSE task provides the relevant patient parameters, which are used to evaluate medication options in the ACTIVATE task.'}})